In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pyhere


In [5]:
dir_data_raw = pyhere.here().resolve().joinpath("data", "raw")
dir_data_interim = pyhere.here().resolve().joinpath("data", "interim")
dir_data_external = pyhere.here().resolve().joinpath("data", "external")

In [12]:
csv_power_plants = pd.read_csv(dir_data_interim/"power_plants_with_generation_transformed.csv", index_col=[0])

In [8]:
df_transformed = pd.read_csv(dir_data_external/"transformed_data_combined_with_nasa.csv", index_col=['index'])
df_transformed

,latitude,longitude,ANN_CLRSKY_SFC_SW_DNI_2009,ANN_CLRSKY_SFC_SW_DNI_2010,ANN_TQV_2009,ANN_TQV_2010,ANN_WS10M_2009,ANN_WS10M_2010,autumn_CLRSKY_SFC_SW_DNI_2009,autumn_CLRSKY_SFC_SW_DNI_2010,...,summer_TQV_2009,summer_TQV_2010,summer_WS10M_2009,summer_WS10M_2010,winter_CLRSKY_SFC_SW_DNI_2009,winter_CLRSKY_SFC_SW_DNI_2010,winter_TQV_2009,winter_TQV_2010,winter_WS10M_2009,winter_WS10M_2010
index,,,,,,,,,,,,,,,,,,,,,
0,32.3220,65.1190,7.72,7.76,9.75,9.77,3.36,3.26,7.440000,8.013333,...,10.276667,11.860000,3.893333,3.386667,6.560000,7.016667,9.003333,9.106667,2.843333,2.963333
1,31.6700,65.7950,7.47,7.36,10.63,10.71,3.22,3.09,7.420000,7.620000,...,11.146667,13.940000,3.183333,3.063333,6.536667,7.096667,10.183333,9.613333,3.196667,3.223333
2,31.6230,65.7920,7.47,7.36,10.63,10.71,3.22,3.09,7.420000,7.620000,...,11.146667,13.940000,3.183333,3.063333,6.536667,7.096667,10.183333,9.613333,3.196667,3.223333
3,34.5560,69.4787,7.96,7.96,8.84,10.03,2.87,2.72,7.240000,7.670000,...,11.420000,15.633333,3.516667,2.890000,7.080000,7.180000,6.540000,6.843333,2.373333,2.356667
4,34.6410,69.7170,7.96,7.96,10.67,12.15,2.39,2.34,7.240000,7.670000,...,15.133333,20.490000,2.380000,2.076667,7.080000,7.180000,7.730000,8.060000,2.070000,2.053333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,-3.2597,-39.2816,7.94,7.94,43.85,42.49,4.88,5.61,7.553333,7.643333,...,49.253333,46.223333,4.293333,5.063333,7.930000,8.150000,36.346667,37.013333,5.480000,6.730000
3006,-27.8970,-50.2862,8.23,8.02,21.05,21.21,1.48,1.55,7.560000,7.370000,...,27.340000,30.793333,1.440000,1.406667,7.246667,6.403333,15.796667,15.886667,1.630000,1.740000
3007,-23.2343,-47.6035,7.21,7.22,33.66,30.67,3.26,3.44,6.873333,7.030000,...,41.926667,42.910000,3.050000,2.956667,6.496667,6.396667,28.103333,21.610000,3.450000,3.810000


In [11]:
df_transformed.columns

Index(['latitude', 'longitude', 'ANN_CLRSKY_SFC_SW_DNI_2009',
       'ANN_CLRSKY_SFC_SW_DNI_2010', 'ANN_TQV_2009', 'ANN_TQV_2010',
       'ANN_WS10M_2009', 'ANN_WS10M_2010', 'autumn_CLRSKY_SFC_SW_DNI_2009',
       'autumn_CLRSKY_SFC_SW_DNI_2010', 'autumn_TQV_2009', 'autumn_TQV_2010',
       'autumn_WS10M_2009', 'autumn_WS10M_2010',
       'spring_CLRSKY_SFC_SW_DNI_2009', 'spring_CLRSKY_SFC_SW_DNI_2010',
       'spring_TQV_2009', 'spring_TQV_2010', 'spring_WS10M_2009',
       'spring_WS10M_2010', 'summer_CLRSKY_SFC_SW_DNI_2009',
       'summer_CLRSKY_SFC_SW_DNI_2010', 'summer_TQV_2009', 'summer_TQV_2010',
       'summer_WS10M_2009', 'summer_WS10M_2010',
       'winter_CLRSKY_SFC_SW_DNI_2009', 'winter_CLRSKY_SFC_SW_DNI_2010',
       'winter_TQV_2009', 'winter_TQV_2010', 'winter_WS10M_2009',
       'winter_WS10M_2010'],
      dtype='object')

In [32]:
csv_power_plants.loc[0:3009, ['capacity_mw', 'primary_fuel_transformed']].index.name = "index"
csv_power_plants.index.rename('index', inplace=True)
csv_power_plants


,capacity_mw,latitude,longitude,primary_fuel_transformed
index,,,,
0,33.0,32.3220,65.1190,Other
1,10.0,31.6700,65.7950,Solar
2,10.0,31.6230,65.7920,Solar
3,66.0,34.5560,69.4787,Other
4,100.0,34.6410,69.7170,Other
...,...,...,...,...
34931,50.0,-12.9667,28.6333,Other
34932,20.0,-12.8167,28.2000,Other
34933,108.0,-17.9167,25.8500,Other


In [34]:
df_transformed_combined = df_transformed.merge(csv_power_plants.loc[0:3009, ['capacity_mw', 'primary_fuel_transformed']],left_on="index", right_on="index")

In [35]:
df_transformed_combined

,latitude,longitude,ANN_CLRSKY_SFC_SW_DNI_2009,ANN_CLRSKY_SFC_SW_DNI_2010,ANN_TQV_2009,ANN_TQV_2010,ANN_WS10M_2009,ANN_WS10M_2010,autumn_CLRSKY_SFC_SW_DNI_2009,autumn_CLRSKY_SFC_SW_DNI_2010,...,summer_WS10M_2009,summer_WS10M_2010,winter_CLRSKY_SFC_SW_DNI_2009,winter_CLRSKY_SFC_SW_DNI_2010,winter_TQV_2009,winter_TQV_2010,winter_WS10M_2009,winter_WS10M_2010,capacity_mw,primary_fuel_transformed
index,,,,,,,,,,,,,,,,,,,,,
0,32.3220,65.1190,7.72,7.76,9.75,9.77,3.36,3.26,7.440000,8.013333,...,3.893333,3.386667,6.560000,7.016667,9.003333,9.106667,2.843333,2.963333,33.00,Other
1,31.6700,65.7950,7.47,7.36,10.63,10.71,3.22,3.09,7.420000,7.620000,...,3.183333,3.063333,6.536667,7.096667,10.183333,9.613333,3.196667,3.223333,10.00,Solar
2,31.6230,65.7920,7.47,7.36,10.63,10.71,3.22,3.09,7.420000,7.620000,...,3.183333,3.063333,6.536667,7.096667,10.183333,9.613333,3.196667,3.223333,10.00,Solar
3,34.5560,69.4787,7.96,7.96,8.84,10.03,2.87,2.72,7.240000,7.670000,...,3.516667,2.890000,7.080000,7.180000,6.540000,6.843333,2.373333,2.356667,66.00,Other
4,34.6410,69.7170,7.96,7.96,10.67,12.15,2.39,2.34,7.240000,7.670000,...,2.380000,2.076667,7.080000,7.180000,7.730000,8.060000,2.070000,2.053333,100.00,Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3005,-3.2597,-39.2816,7.94,7.94,43.85,42.49,4.88,5.61,7.553333,7.643333,...,4.293333,5.063333,7.930000,8.150000,36.346667,37.013333,5.480000,6.730000,18.90,Wind
3006,-27.8970,-50.2862,8.23,8.02,21.05,21.21,1.48,1.55,7.560000,7.370000,...,1.440000,1.406667,7.246667,6.403333,15.796667,15.886667,1.630000,1.740000,6.50,Other
3007,-23.2343,-47.6035,7.21,7.22,33.66,30.67,3.26,3.44,6.873333,7.030000,...,3.050000,2.956667,6.496667,6.396667,28.103333,21.610000,3.450000,3.810000,2.76,Other
